SMT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SMT version.

Necessary libraries:

In [ ]:
!pip install z3-solver

Necessary imports:

In [2]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [3]:
# open the file in Instances folder
f = open("inst10.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 10
items: 13
load_size: [185, 190, 200, 180, 200, 190, 200, 180, 195, 190]
item_size: [22, 17, 10, 8, 14, 12, 17, 19, 25, 25, 6, 21, 6]
distance:
 [[  0  21  86  14  84  72  24  54  83  70   8  91  42  57]
 [ 21   0  71  35  70  51  16  75  62  91  29  70  57  52]
 [ 86  71   0 100  39  70  87 137  81  73  78 103 128  33]
 [ 14  35 100   0  98  86  38  49  97  56  22 105  29  71]
 [ 84  70  39  98   0 109  86 135 120  57  76 129 126  27]
 [ 63  51  70  77 109   0  49 117  11 133  71  64  90 103]
 [ 24  16  87  38  86  49   0  78  60  94  32  67  41  60]
 [ 63  84 137  49 135 135  87   0 146  79  59 154  65 108]
 [ 74  62  81  88 120  11  60 128   0 144  82  68  94 114]
 [ 70  91  73  56  57 142  94  79 153   0  63 161  72  40]
 [  8  29  78  22  76  80  32  59  91  63   0  99  50  49]
 [ 91  70 133 105 129  64  67 145  68 161  99   0  91 122]
 [ 42  57 128  29 126  90  41  65  94  72  50  91   0  99]
 [ 57  52  33  71  27 103  60 108 114  40  49 122  99   0]]


Other variables:

In [4]:
max_path_length = n-(m-1)
paths = [[Int("p_%s_%s" % (i,j)) for j in range(max_path_length)] for i in range(m)] 
# create a matrix of mxn boolean variables for the assignment of items to couriers
assignment = [[Bool("a_%s_%s" % (i,j)) for j in range(n)] for i in range(m)]
# Create a solver instance
s = Solver()


Main constraints:

In [5]:
constraints = []
constraints.append(n>=m)

# Each item has a size > 0 and <= max(load size)
for i in range(n):
    constraints.append(And(item_size[i] > 0, item_size[i] <= max(load_size)))

# Each item is delivered at most by one courier, and at least by one courier
for i in range(n):
    constraints.append(Sum([If(paths[c][j] == i, 1, 0) for c in range(m) for j in range(max_path_length)]) == 1)

# assignement constraints 
for c in range(m):
    for i in range(n):
        constraints.append(assignment[c][i] == Or([paths[c][j] == i for j in range(max_path_length)]))

# Each courier can carry at most its load size
for c in range(m):
    constraints.append(Sum([If(assignment[c][i], item_size[i], 0) for i in range(n)]) <= load_size[c])

# Each courier must deliver at least one item
for c in range(m):
    constraints.append(Sum([If(assignment[c][i], 1, 0) for i in range(n)]) >= 1)

best_max_distance = math.inf
s.add(constraints)
courier_distances = [[0] for c in range(m)]
courier_loads = [[0] for c in range(m)]
best_courier_distances = [[0] for c in range(m)]
break_counter = 0
loop_counter = 0
for l in range(1000):
    if s.check() == sat:
        loop_counter += 1
        model = s.model()
        # get the values of the paths
        paths_values = [[model[paths[i][j]] for j in range(max_path_length)] for i in range(m)]
        # get path for each courier as a list of items, taking only the values in the range [0,n-1]
        paths_items = [[paths_values[i][j].as_long() for j in range(max_path_length) if paths_values[i][j].as_long() < n] for i in range(m)]
        paths_items = [list(filter(lambda x: x != -1, paths_items[i])) for i in range(m)] 
        # get the total distance for each courier by adding also the distance from the depot to the first item and from the last item to the depot
        for c in range(m):
            dist = distance[n][paths_items[c][0]] + distance[paths_items[c][-1]][n]
            for i in range(len(paths_items[c])-1):
                dist += distance[paths_items[c][i]][paths_items[c][i+1]]
            courier_distances[c][0] = dist

        # only accept solutions with max distance < best_max_distance
        # and update best_max_distance
        max_distance = max([courier_distances[c][0] for c in range(m)])
        if max_distance < best_max_distance:
            best_max_distance = max_distance
            break_counter = loop_counter
            best_courier_distances = courier_distances.copy()
            print("max_distance:", max_distance)
            print("paths_items:", paths_items)
            print("courier_distances:", courier_distances)
        else:
            if loop_counter - break_counter > 200:
                break
    else:
        break


max_distance: 306
paths_items: [[3], [10], [11], [2, 7, 1], [5], [12], [6, 4], [9], [0], [8]]
courier_distances: [[142], [98], [244], [306], [206], [198], [173], [80], [114], [228]]
max_distance: 270
paths_items: [[7], [9], [11, 10], [0], [2], [3], [4, 5], [6, 8], [12], [1]]
courier_distances: [[216], [80], [270], [114], [66], [142], [239], [234], [198], [104]]
max_distance: 246
paths_items: [[3], [5], [10, 12], [4, 2], [11], [1], [9], [0], [6, 7], [8]]
courier_distances: [[142], [206], [198], [99], [244], [104], [80], [114], [246], [228]]
max_distance: 244
paths_items: [[0], [7], [1, 3], [11], [9], [8, 2], [10], [6], [4, 5], [12]]
courier_distances: [[114], [216], [158], [244], [80], [228], [98], [120], [239], [198]]
